# Linear models: Support Vector Machines (SVM)

In this notebook we are going to explore linear models and Support Vector Machines (SVM in short).

Let's first import the required packages.

In [ ]:
# -- put here your ID number (numero di matricola)
ID_number = 

from sklearn import datasets, preprocessing, linear_model, svm
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np

In [ ]:
## SVM for linearly separable data

Let's start by creating a simple linearly separable dataset for binary classification, where the instance space is $\mathcal{X} =\mathbb{R}^2$ (so that we can visualize it). Just to make things easier, we are going to rescale it too.

In [ ]:
X, y = datasets.make_blobs(n_samples = 500, centers = 2, n_features = 2, random_state=ID_number)

scaler = preprocessing.StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

The following code plots the dataset, it is useful for later parts too.

In [ ]:
plt.title("Plot of dataset")
plt.scatter(X[:, 0], X[:, 1], c=y);

Now let's run the perceptron, using $\texttt{linear\_model.Perceptron(...)}$ from sklearn. We fix the number of iterations to 100 so that it runs quickly (since we have a pretty simple to classifiy and linearly separable dataset) , and $\texttt{random\_state=10}$.

What do we expect in terms of training error? 

In [ ]:
# -- Create a perceptron classifier
model_perceptron_1 = linear_model.Perceptron(max_iter=100, random_state = 10)

# -- Train the model on all data
model_perceptron_1.fit(X, y)

# -- Get the training error as 1 - score()
training_error = 1 - model_perceptron_1.score(X,y)

# -- Print the training error
print("Training error: ", training_error)

The following code plots the *decision boundary* of a model and the training set. It is useful for later parts too.

In [ ]:
# --- model_perceptron should be already trained
def plot_perceptron_boundaries(X, y, model_perceptron):

    plt.scatter(X[:, 0], X[:, 1], c=y, s=30)
    ax = plt.gca()
    plt.title("Plot of perceptron decision boundary")
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    # -- create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = model_perceptron.decision_function(xy).reshape(XX.shape)
    # -- plot decision boundary and margins
    ax.contour(XX, YY, Z, colors='k', levels=[0], alpha=1, linestyles=['-']);

In [ ]:
# -- let's print the decision boundaries of model_perceptron_1
plot_perceptron_boundaries(X, y, model_perceptron_1)

If we change the value of $\texttt{random\_state}$ in the perceptron, it will start from a different model. 

Let's run the perceptron with different $\texttt{random\_state}$. How will the solution compare to the above?

In [ ]:
# -- Create a perceptron classifier
model_perceptron_2 = linear_model.Perceptron(max_iter=100, random_state = 12)

# -- Training the model
model_perceptron_2.fit(X, y)

# -- Get the training error as 1 - score()
training_error = 1 - model_perceptron_2.score(X,y)

# -- Print the training error
print("Training error: ", training_error)

What about the decision boundary? Let's plot it.

In [ ]:
# -- let's print the decision boundaries of model_perceptron_2
plot_perceptron_boundaries(X, y, model_perceptron_2)

Which model is better? 

Is any of these the *best* choice?

Now, let's run the hard-SVM on the same data. To obtain (an almost) hard-SVM in sklearn, we can use $\texttt{svm.SVC(...)}$ with a very high value of the parameter $C$.

Note: the $C$ parameter used by $\texttt{svm.SVC(...)}$ method in sklearn is approximately equal to $1 / \lambda$, with respect to our use and definition of $\lambda$. 

In [ ]:
# -- Creating a SVM model
model_svm_1 = svm.SVC(kernel="linear", C=1e8)

# -- Training the model
model_svm_1.fit(X, y)

# -- Get the training error as 1 - score()
training_error = 1 - model_svm_1.score(X,y)

# -- Print the training error
print("Training error: ", training_error)

Plot the SVM decision boundary.

In [ ]:
# -- Code for plotting the decision boundary for svm
# -- svm_models must be a list of svm models
def plot_svm_boundaries(X, y, svm_models, show_sv=False, show_margin=False):

    plt.scatter(X[:, 0], X[:, 1], c=y, s=30)
    ax = plt.gca()
    plt.title("Plot of SVM decision boundary")
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T

    palette = [f'C{i}' for i in range(len(svm_models))]
    handles = []

    for idx, svm in enumerate(svm_models):
    
        Z = svm.decision_function(xy).reshape(XX.shape)
        # plot decision boundary and margins
        if show_margin:
            ax.contour(XX, YY, Z, colors = palette[idx], levels=[-1, 0, 1], alpha=0.5, linestyles=["--", "-", "--"])
        else:
            ax.contour(XX, YY, Z, colors = palette[idx], levels=[0], alpha=1, linestyles=['-']);
        handles.append(Line2D([0], [0], label=f'SVM {idx+1}', color=palette[idx]))
        if show_sv:
            plt.scatter(svm.support_vectors_[:, 0], svm.support_vectors_[:, 1], color = palette[idx], marker='s', alpha=.85)

    ax.legend(handles = handles, loc = 'lower left')


In [ ]:
plot_svm_boundaries(X, y, [model_svm_1])

Let's see what the support vectors are. They are defined in attribute $\texttt{support\_vectors\_}$

In [ ]:
# -- print the support vectors
print('Support Vectors:\n', model_svm_1.support_vectors_)

print('---')

# -- print dual coefficients
print('Dual Coefficients of Support Vectors:\n', model_svm_1.dual_coef_)

Let's see what happens moving one support vector. We first obtain the indices of the support vectors.

In [ ]:
# -- let's actually plot the support vectors
plot_svm_boundaries(X, y, [model_svm_1], show_sv = True)

---

Now, let's try to play a bit with the support vectors, in order to see how the svm model is going to behave.

For instance, let's try to move one of the above support vectors.

In [ ]:
# -- print the indices of support vectors (attribute support)
print(model_svm_1.support_)
# -- for example, let's move the point indexed by 321, that is
X[321]

Now let's move one support vector closer to the points in the same class.

In [ ]:
# -- let's copy the data and move one support vector close to the points in the same class
X1 = X.copy()
# the suppport vector indexed by 321 goes from x coordinate ~ -0.2 x coordinate -1
X1[321, 0] = -1

# -- let's plot the new dataset
plt.title("Plot of dataset")
plt.scatter(X1[:, 0], X1[:, 1], c=y)

# -- see the movement of the support vector (then comment to see the whole actual dataset)
plt.scatter(X[321, 0], X[321, 1], color = 'red', edgecolor='k')
plt.scatter(X1[321, 0], X1[321, 1], color = 'red', edgecolor='k')
plt.annotate('', xytext=(X[321, 0], X[321, 1]), xy=(X1[321, 0], X1[321, 1]),
            arrowprops=dict(facecolor='red', shrink=0.07, width=2.5));

Let's run the SVM on the new data.

In [ ]:
# -- Creating a SVM model
model_svm_2 = svm.SVC(kernel="linear", C=1e8)

# -- Training the model
model_svm_2.fit(X1, y)

# -- Get the training error as 1 - score()
training_error = 1 - model_svm_2.score(X1,y)

# -- Print the training error
print("Training error: ", training_error)

Plot the SVM decision boundary and the previous decision boundary.

In [ ]:
# -- Plot boundaries for different SVMs
plot_svm_boundaries(X1, y, [model_svm_1, model_svm_2])

In [ ]:
# -- Let's see now what the support vectors are
print('Support Vectors of model 1:\n', model_svm_1.support_vectors_)
print('---')
print('Support Vectors of model 2:\n', model_svm_2.support_vectors_)
print('---')
# -- Let's print the dual coefficients as well
print('Dual Coefficients of Support Vectors of model 1:\n', model_svm_1.dual_coef_)
print('---')
print('Dual Coefficients of Support Vectors of model 2:\n', model_svm_2.dual_coef_)

In [ ]:
# -- Plot svm boundaries for comparison with support vectors highlighted
plot_svm_boundaries(X1, y, [model_svm_1, model_svm_2], show_sv = True)

---

Now let's move the same support vector to the right, i.e., closer to the points in the other class.

In [ ]:
# -- let's copy the original data and move one support vector close to the points in the other class
X2 = X.copy()
X2[321, 0] = 0.2

# -- let's plot the new dataset
plt.title("Plot of dataset")
plt.scatter(X2[:, 0], X2[:, 1], c=y)

# -- see the movement of the support vector (then comment to see the whole actual dataset)
plt.scatter(X[321, 0], X[321, 1], color = 'red', edgecolor='k')
plt.scatter(X2[321, 0], X2[321, 1], color = 'red', edgecolor='k')
plt.annotate('', xytext=(X[321, 0], X[321, 1]), xy=(X2[321, 0], X2[321, 1]),
            arrowprops=dict(facecolor='red', shrink=0.12, width=2.5, headwidth=10.0));

Let's run the SVM on the new data.

In [ ]:
# -- Creating a SVM model
model_svm_3 = svm.SVC(kernel="linear", C=1e8)

# -- Training the model
model_svm_3.fit(X2, y)

# -- Get the training error as 1 - score()
training_error = 1 - model_svm_3.score(X2,y)

# -- Print the training error
print("Training error: ", training_error)

In [ ]:
# -- Plot svm boundaries
plot_svm_boundaries(X2, y, [model_svm_1, model_svm_2, model_svm_3])

Let's plot the new decision boundary, and the old ones too.

In [ ]:
# -- Let's see now what the support vectors are
print('Support Vectors of model 1:\n', model_svm_1.support_vectors_)
print('---')
print('Support Vectors of model 2:\n', model_svm_2.support_vectors_)
print('---')
print('Support Vectors of model 3:\n', model_svm_3.support_vectors_)

In [ ]:
# -- Let's print the dual coefficients as well
print('Dual Coefficients of Support Vectors of model 1:\n', model_svm_1.dual_coef_)
print('---')
print('Dual Coefficients of Support Vectors of model 2:\n', model_svm_2.dual_coef_)
print('---')
print('Dual Coefficients of Support Vectors of model 3:\n', model_svm_3.dual_coef_)

In [ ]:
# -- Plot svm boundaries with support vectors for comparison
plot_svm_boundaries(X1, y, [model_svm_1, model_svm_2, model_svm_3], show_sv = True)

## SVM for non-linearly separable data

Let's make a dataset that is not linearly separble, and let's plot it.

In [ ]:
X_nls, y_nls = datasets.make_blobs(n_samples = 500, centers = 2, n_features = 2, random_state = ID_number)

scaler.fit(X_nls)
X_nls = scaler.transform(X_nls)

# -- let's manually create a nls dataset
a = np.array([[0.3, 2]])
b = np.array([0])
X_nls = np.concatenate((X_nls, a))
y_nls = np.concatenate((y_nls, b))
a = np.array([[0.1, -1.5]])
b = np.array([0])
X_nls = np.concatenate((X_nls, a))
y_nls = np.concatenate((y_nls, b))

a = np.array([[0, 0.1]])
b = np.array([1])
X_nls = np.concatenate((X_nls, a))
y_nls = np.concatenate((y_nls, b))

plt.title("Plot of dataset")
plt.scatter(X_nls[:, 0], X_nls[:, 1], c=y_nls);

Let's try to learn a hard-SVM. It means that the parameter C, which is approximately equal to $1/\lambda$ with $\lambda$ as in our slides.

In [ ]:
# -- Creating a HARD SVM model
model_hard_svm_1 = svm.SVC(kernel="linear", C=1e8)

# -- Training the model
model_hard_svm_1.fit(X_nls, y_nls)

# -- Get the training error as 1 - score()
training_error = 1 - model_hard_svm_1.score(X_nls, y_nls)

# -- Print the training error
print("Training error: ", training_error)

The following code plots the decision boundary, as well as the margin.

In [ ]:
# -- Plot of hard SVM decision boundary
plot_svm_boundaries(X_nls, y_nls, [model_hard_svm_1], show_sv=True, show_margin=True)

In [ ]:
# -- Let's see now what the support vectors and dual coefficients are
print('Support Vectors of model 1:\n', model_hard_svm_1.support_vectors_)
print('---')
print('Dual Coefficients of Support Vectors of model 1:\n', model_hard_svm_1.dual_coef_)

Let's try with a smaller value of C ($10^4$), that corresponds to larger value of $\lambda$.

What do you expect?

In [ ]:
# -- Creating a soft SVM model
model_soft_svm_2 = svm.SVC(kernel="linear", C=1e4)

# -- Training the model
model_soft_svm_2.fit(X_nls, y_nls)

# -- Get the training error as 1 - score()
training_error = 1 - model_soft_svm_2.score(X_nls, y_nls)

# -- Print the training error
print("Training error: ", training_error)

What about the decision boundary and the margin?

In [ ]:
# -- Plot of soft SVM decision boundary
plot_svm_boundaries(X_nls, y_nls, [model_soft_svm_2], show_sv=True, show_margin=True)

In [ ]:
# -- Let's see now what the support vectors and dual coefficients are
print('Support Vectors of model 1:\n', model_hard_svm_1.support_vectors_)
print('---')
print('Support Vectors of model 2:\n', model_soft_svm_2.support_vectors_)

In [ ]:
print('Dual Coefficients of Support Vectors for model 1:\n', model_hard_svm_1.dual_coef_)
print('---')
print('Dual Coefficients of Support Vectors for model 2:\n', model_soft_svm_2.dual_coef_)

Let's repeat everything for an even smaller C, i.e., C = 100

In [ ]:
# -- Creating a soft SVM model
model_soft_svm_3 = svm.SVC(kernel="linear", C=100)

# -- Training the model
model_soft_svm_3.fit(X_nls, y_nls)

# -- Get the training error as 1 - score()
training_error = 1 - model_soft_svm_3.score(X_nls, y_nls)

# -- Print the training error (seems to be exactly the same as before)
print("Training error: ", training_error)

In [ ]:
# -- Plot of soft SVM decision boundary
plot_svm_boundaries(X_nls, y_nls, [model_soft_svm_3], show_sv=True, show_margin=True)

What about setting C = 1?

In [ ]:
# -- Creating a soft SVM model 
model_soft_svm_4 = svm.SVC(kernel="linear", C=1)

# -- Training the model
model_soft_svm_4.fit(X_nls, y_nls)

# -- Get the training error as 1 - score()
training_error = 1 - model_soft_svm_4.score(X_nls, y_nls)

# -- Print the training error (corresponds to 3 misclassified points)
print("Training error: ", training_error)

In [ ]:
# -- Plot of soft SVM decision boundary
plot_svm_boundaries(X_nls, y_nls, [model_soft_svm_4], show_sv=True, show_margin=True)

Let's see what are the support vectors.

In [ ]:
# -- Let's see now what the support vectors and dual coefficients are
print('Support Vectors of model 4:\n\n', model_soft_svm_4.support_vectors_)
print('---')
print('Dual Coefficients of Support Vectors for model 4:\n\n', model_soft_svm_4.dual_coef_)

### Comparison with Perceptron

Just for comparison, let's run the perceptron on the same dataset with various initial random states

Let's plot the decision boundary.

In [ ]:
model_perceptron_1 = linear_model.Perceptron(max_iter=100, random_state = 143)
model_perceptron_1.fit(X_nls, y_nls)
training_error = 1 - model_perceptron_1.score(X_nls, y_nls)
print("Training error: ", training_error)
# -- plot perceptron boundaries
plot_perceptron_boundaries(X_nls, y_nls, model_perceptron_1)

In [ ]:
model_perceptron_2 = linear_model.Perceptron(max_iter=100, random_state = 10)
model_perceptron_2.fit(X_nls, y_nls)
training_error = 1 - model_perceptron_2.score(X_nls, y_nls)
print("Training error: ", training_error)
# -- plot perceptron boundaries
plot_perceptron_boundaries(X_nls, y_nls, model_perceptron_2)

In [ ]:
model_perceptron_3 = linear_model.Perceptron(max_iter=100, random_state = 24)
model_perceptron_3.fit(X_nls, y_nls)
training_error = 1 - model_perceptron_3.score(X_nls, y_nls)
print("Training error: ", training_error)
# -- plot perceptron boundaries
plot_perceptron_boundaries(X_nls, y_nls, model_perceptron_3)

### K-Fold Cross Validation

Now, let's try to select the best SVM model using $k$-fold cross validation, with respect to the parameter $C$.

More specifically, let's fix a non-linearly separable dataset. You need to perform train-validation-test split, fix a grid for the hyperparameter $C$, and perform model selection by running $k$-fold cross validations (let's say $k = 5$, but you can try to change).

At the end, we would like to collect the best models, in terms of generalization error, achieved by you. 

In [ ]:
# -- generate the dataset
X, Y = datasets.make_blobs(n_samples = 1000, centers = 2, n_features = 2, 
                            center_box=(-7.5, 7.5), random_state = 37, cluster_std = 2.8)

# -- divide in: train_and_val = 5/6, test = 1/6

# -- TO COMPLETE (from now on)
# -- perform train-val-test split
# -- fix k of k-fold = 5 (for example)

m = 
m_train_and_val = 
m_test = 

# -- print sizes
print("Amount of data for training and validation:", m_train_and_val)
print("Amount of data for test:", m_test)

### Comparison with Perceptron

Finally, let's compare the best model obtained above with some perceptrons. 

In [ ]:
# -- TO COMPLETE
perceptron_1 = 

In [ ]:
# -- TO COMPLETE: compute generalization error

In [1]:
# -- you may try other perceptron models with different seeds